# Google Cloud SQL Connection Example

This notebook demonstrates how to connect to a Cloud SQL MySQL instance and run various queries.

## Instructions
1. Run the installation cell first
2. Authenticate with Google Cloud
3. Update the configuration with your values
4. Run the query examples

## 1. Install Dependencies

In [ ]:
!pip install -q sqlalchemy pymysql cloud-sql-python-connector google-auth pandas matplotlib

## 2. Authenticate with Google Cloud

In [ ]:
from google.colab import auth
auth.authenticate_user()

# Verify authentication
!gcloud config list

## 3. Configuration

**UPDATE THESE VALUES** with your Cloud SQL instance details.

In [ ]:
# Configuration - UPDATE THESE VALUES
PROJECT_ID = "your-project-id"
REGION = "us-central1"
INSTANCE_NAME = "employees-mysql-instance"
DB_USER = "test-user"
DB_PASSWORD = "TestPass123#"  # Use Secret Manager in production!
DATABASE = "employees"

print(f"Configuration:")
print(f"  Project: {PROJECT_ID}")
print(f"  Instance: {INSTANCE_NAME}")
print(f"  Database: {DATABASE}")

## 4. Connect to Cloud SQL

Using the Cloud SQL Python Connector for secure connection.

In [ ]:
from google.cloud.sql.connector import Connector
from sqlalchemy import create_engine
import pandas as pd

# Initialize Connector
connector = Connector()

def getconn():
    conn = connector.connect(
        f"{PROJECT_ID}:{REGION}:{INSTANCE_NAME}",
        "pymysql",
        user=DB_USER,
        password=DB_PASSWORD,
        db=DATABASE
    )
    return conn

# Create SQLAlchemy engine
engine = create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

print("Connected to Cloud SQL successfully!")

## 5. Test Query

In [ ]:
# Simple test query
query = "SELECT COUNT(*) as total_employees FROM employees"
df = pd.read_sql(query, engine)
print(df)

## 6. Employee Count by Department

In [ ]:
import matplotlib.pyplot as plt

query = '''
SELECT
    d.dept_name,
    COUNT(de.emp_no) as employee_count
FROM departments d
JOIN dept_emp de ON d.dept_no = de.dept_no
WHERE de.to_date = '9999-01-01'
GROUP BY d.dept_name
ORDER BY employee_count DESC
'''

df = pd.read_sql(query, engine)
print("Employees by Department:")
print(df)

# Create a simple bar chart
plt.figure(figsize=(10, 6))
plt.barh(df['dept_name'], df['employee_count'])
plt.xlabel('Number of Employees')
plt.ylabel('Department')
plt.title('Current Employees by Department')
plt.tight_layout()
plt.show()

## 7. Average Salary by Department

In [ ]:
query = '''
SELECT
    d.dept_name,
    ROUND(AVG(s.salary), 2) as avg_salary
FROM departments d
JOIN dept_emp de ON d.dept_no = de.dept_no
JOIN salaries s ON de.emp_no = s.emp_no
WHERE de.to_date = '9999-01-01'
AND s.to_date = '9999-01-01'
GROUP BY d.dept_name
ORDER BY avg_salary DESC
'''

df_salary = pd.read_sql(query, engine)
print("Average Salary by Department:")
print(df_salary)

# Visualize
plt.figure(figsize=(10, 6))
plt.barh(df_salary['dept_name'], df_salary['avg_salary'])
plt.xlabel('Average Salary ($)')
plt.ylabel('Department')
plt.title('Average Salary by Department')
plt.tight_layout()
plt.show()

## 8. Employee Demographics

In [ ]:
query = '''
SELECT
    gender,
    COUNT(*) as count
FROM employees
GROUP BY gender
'''

df_gender = pd.read_sql(query, engine)
print("Employee Demographics:")
print(df_gender)

# Pie chart
plt.figure(figsize=(8, 8))
plt.pie(df_gender['count'], labels=df_gender['gender'], autopct='%1.1f%%', startangle=90)
plt.title('Employee Gender Distribution')
plt.axis('equal')
plt.show()

## 9. Hire Date Distribution

In [ ]:
query = '''
SELECT
    YEAR(hire_date) as hire_year,
    COUNT(*) as hires
FROM employees
GROUP BY YEAR(hire_date)
ORDER BY hire_year
'''

df_hires = pd.read_sql(query, engine)
print("Hires by Year:")
print(df_hires.head(10))

# Line chart
plt.figure(figsize=(12, 6))
plt.plot(df_hires['hire_year'], df_hires['hires'], marker='o')
plt.xlabel('Year')
plt.ylabel('Number of Hires')
plt.title('Employee Hires Over Time')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Current Managers with Details

In [ ]:
query = '''
SELECT
    e.emp_no,
    CONCAT(e.first_name, ' ', e.last_name) as manager_name,
    d.dept_name,
    s.salary as current_salary,
    dm.from_date as manager_since,
    YEAR(CURDATE()) - YEAR(dm.from_date) as years_as_manager
FROM dept_manager dm
JOIN employees e ON dm.emp_no = e.emp_no
JOIN departments d ON dm.dept_no = d.dept_no
JOIN salaries s ON e.emp_no = s.emp_no
WHERE dm.to_date = '9999-01-01'
AND s.to_date = '9999-01-01'
ORDER BY d.dept_name
'''

df_managers = pd.read_sql(query, engine)
print("Current Department Managers:")
print(df_managers)

## 11. Salary Statistics

In [ ]:
query = '''
SELECT
    COUNT(*) as employee_count,
    MIN(salary) as min_salary,
    MAX(salary) as max_salary,
    ROUND(AVG(salary), 2) as avg_salary,
    ROUND(STDDEV(salary), 2) as stddev_salary,
    ROUND(AVG(salary) - STDDEV(salary), 2) as lower_bound,
    ROUND(AVG(salary) + STDDEV(salary), 2) as upper_bound
FROM salaries
WHERE to_date = '9999-01-01'
'''

df_stats = pd.read_sql(query, engine)
print("Current Salary Statistics:")
print(df_stats.T)  # Transpose for better readability

## 12. Salary Distribution Histogram

In [ ]:
query = "SELECT salary FROM salaries WHERE to_date = '9999-01-01'"
df_all_salaries = pd.read_sql(query, engine)

plt.figure(figsize=(12, 6))
plt.hist(df_all_salaries['salary'], bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Salary ($)')
plt.ylabel('Frequency')
plt.title('Current Salary Distribution')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Statistics:")
print(df_all_salaries['salary'].describe())

## 14. Clean Up Connection

Run this cell when you're done to close the connection.

In [ ]:
# Close the connection when done
engine.dispose()
connector.close()
print("Connection closed")